In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import re
import datetime
import pylab as pl
pl.style.use('fivethirtyeight')
import statsmodels.formula.api as smf
from statsmodels.graphics.api import abline_plot
import statsmodels.api as sma

In [3]:
def convertDate(d):
    date = datetime.datetime.strptime(d, '%m/%d/%Y %I:%M:%S %p')
    return datetime.date(date.year, date.month, date.day)

In [4]:
complaints = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', converters={'Created Date': convertDate})

In [5]:
complaints = complaints.rename(columns = {'Created Date': 'Date'})
complaints['dow'] = complaints.apply(lambda x: x['Date'].weekday(), axis=1)
complaints['month'] = complaints.apply(lambda x: x['Date'].month, axis=1)

In [6]:
def convertStreak(streak):
    # N is post season?
    #if streak == "N":
    #    return float('Nan')
    if not streak:
        return float('Nan')
    sign = -1 if streak[0] == "-" else 1
    return len(streak) * sign

def getDateConverter(year):
    def convertDate(date):
        # Handle duplicate headers
        if date == 'Date':
            return None
        
        # Remove double-header indication (1), (2) and convert to date
        d = datetime.datetime.strptime(re.sub(r'\s\(.*\)', '', date) + ' ' + str(year), '%A %b %d %Y')
        #d.year = year
        return datetime.date(d.year, d.month, d.day)
    
    return convertDate

def convertWL(wl):
    # Covert the W/L column to numeric
    if wl and wl[0] == 'W':
        return 1
    return 0

def readGameLog(csv, convertDate):
    df = pd.read_csv(csv, skip_blank_lines=True, converters={'Streak': convertStreak, 'Date': convertDate, 'W/L': convertWL})
    
    # Remove the headers in the middle of the file and postseason
    df = df.drop(df.index[
        (df.Rk == "Rk") # Drop header rows
    ])
    
    #print [d.days for d in np.diff(df.Date)]
    
    # First game in double headers
    df = df.drop(df.index[(np.append([d.days for d in np.diff(df.Date)], [1]) == 0)])
    
    return df

In [63]:
y = []
m = []
years = range(2010,2016)
for year in years:
    y.append(readGameLog("teams_NYY_%s-schedule-scores_team_schedule.csv" % str(year), getDateConverter(year)))
    m.append(readGameLog("teams_NYM_%s-schedule-scores_team_schedule.csv" % str(year), getDateConverter(year)))
    
yankees = pd.concat(y, ignore_index=True)
yankees['y_postseason'] = np.isnan(yankees.Rk.astype('float'))
yankees = yankees.rename(columns = {'Streak': 'y_streak'})
mets = pd.concat(m, ignore_index=True)
mets['m_postseason'] = np.isnan(mets.Rk.astype('float'))
mets = mets.rename(columns = {'Streak': 'm_streak'})

In [64]:
baseball = pd.merge(yankees, mets, on='Date', how='outer', sort=True)
#baseball = pd.merge(yankees, mets, on='Date', how='inner', sort=True)

In [65]:
baseball['m_postseason'] = baseball['m_postseason'].astype(float)
baseball['y_postseason'] = baseball['y_postseason'].astype(float)

In [67]:
baseball['y_postseason'].head(200)

0       0
1     NaN
2       0
3       0
4     NaN
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14    NaN
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
170     0
171     0
172     1
173     1
174     1
175     1
176     1
177     1
178     1
179     1
180     1
181     0
182   NaN
183     0
184     0
185     0
186     0
187   NaN
188     0
189     0
190     0
191     0
192   NaN
193     0
194     0
195     0
196     0
197     0
198     0
199     0
Name: y_postseason, dtype: float64

In [68]:
merged = pd.merge(baseball, complaints, on='Date')

In [136]:
def interp(a):
    if np.isnan(a[0]):
        a[0] = 0.
        
    for i in range(len(a)):
        if np.isnan(a[i]):
            a[i] = a[i-1]

def doStuff(df):
    grouped = df.groupby('Date')
    
    # Get numbers for each day
    ystreak = []
    ypost = []
    mstreak = []
    mpost = []
    counts = []
    dow = []
    month = []
    for d in sorted(grouped.groups.keys()):
        group = grouped.get_group(d)
        mean = group.mean()
        
        ystreak.append(mean['y_streak'])
        ypost.append(group['y_postseason'].iloc[0])
        
        mstreak.append(mean['m_streak'])
        mpost.append(group['m_postseason'].iloc[0])
        
        dow.append(mean['dow'])
        counts.append(grouped.get_group(d).count()['Date'])
        month.append(mean['month'])
        
    # Detrend number of complaints
    cycle, trend = sma.tsa.filters.hpfilter(counts, lamb = 104976000000)
    
    # Create a single postseason variable
    # 0 is not postseason, 1 is yankees in post, 2 is mets in post
    # and 3 is both in post.
    q = np.array(mpost)*2
    r = np.array(ypost)
    q[np.isnan(q)] = 0
    r[np.isnan(r)] = 0
    post = q+r
    
    interp(ystreak)
    interp(mstreak)
    
    # Run Regression vs. Everything
    mod = smf.ols(formula='c ~ C(d) + C(m) + ys + ms + C(p)', data = {
        'c': cycle, 
        'ys': ystreak, 'yp': ypost,
        'ms': mstreak, 'mp': mpost,
        'd': dow, 'm': month,
        'p': post
    }).fit()
    print mod.summary()

In [137]:
doStuff(merged)

                            OLS Regression Results                            
Dep. Variable:                      c   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     179.1
Date:                Mon, 09 Nov 2015   Prob (F-statistic):          1.13e-304
Time:                        00:23:37   Log-Likelihood:                -8446.3
No. Observations:                1070   AIC:                         1.693e+04
Df Residuals:                    1051   BIC:                         1.703e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept      788.6730    464.752      1.697   

In [138]:
noise = merged[merged['Complaint Type'].str.contains('Noise')]

In [139]:
doStuff(noise)

                            OLS Regression Results                            
Dep. Variable:                      c   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     45.04
Date:                Mon, 09 Nov 2015   Prob (F-statistic):          6.58e-117
Time:                        00:24:30   Log-Likelihood:                -6519.2
No. Observations:                1070   AIC:                         1.308e+04
Df Residuals:                    1051   BIC:                         1.317e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept     -178.1546     76.751     -2.321   

In [140]:
wonoise = merged[~merged['Complaint Type'].str.contains('Noise')]

In [141]:
doStuff(wonoise)

                            OLS Regression Results                            
Dep. Variable:                      c   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     205.0
Date:                Mon, 09 Nov 2015   Prob (F-statistic):               0.00
Time:                        00:26:28   Log-Likelihood:                -8448.9
No. Observations:                1070   AIC:                         1.694e+04
Df Residuals:                    1051   BIC:                         1.703e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept      966.8276    465.898      2.075   